In [ ]:
#INSTALACION DE PAQUETES NECESARIOS
# Este script instala los paquetes necesarios para el análisis de datos y visualización en Python.
pip install pandas matplotlib seaborn folium numpy



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
# LLIMPIEZA DE DATASET Y MEJORAS
import pandas as pd

# Cargar el dataset original
df=pd.read_csv('../data/accidentes.csv', encoding='latin1')

# Mejoras del dataset
# 1. Eliminar columnas innecesarias
df= df.drop(columns=["Codigo_Accidente"])
# 2. Limpieza de datos
df["Fecha_Ocurrencia"]=pd.to_datetime(df["Fecha_Ocurrencia"], errors='coerce')
# Generar nuevas columnas de año, mes y día
df["Día"]=df["Fecha_Ocurrencia"].dt.day
df["Mes"]=df["Fecha_Ocurrencia"].dt.month
df["Año"]=df["Fecha_Ocurrencia"].dt.year

# Cambiar el valor NO APLICA de la columna por 0
df['Muertes']=df['Muertes'].replace('NO APLICA', 0)

# Cambiar la palabra SIN INFORMACION DEL BARRIO por No registra y de igual manera tambien la palabra VILLAVICENCIO por No registra
df['Barrio']=df['Barrio'].replace('SIN INFORMACIÃN DEL BARRIO', 'No registra')
df['Barrio']=df['Barrio'].replace('VILLAVICENCIO', 'No registra')
df['Barrio']=df['Barrio'].replace('VÃA GRANADA-VILLAVICENCIO', 'Afueras de la ciudad')


# Crear Columna de Mortalidad
df['Mortalidad'] = df['Muertes'].astype(int) > 0
# Crear columna con etiquetas legibles
df['Mortalidad_label'] = df['Mortalidad'].map({True: "Con mortalidad", False: "Sin mortalidad"})

# Guardar el dataset mejorado
df.to_csv('../data/accidentesLimpio.csv', index=False, encoding='utf-8')

/tmp/ipykernel_7047/3051703296.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Fecha_Ocurrencia"]=pd.to_datetime(df["Fecha_Ocurrencia"], errors='coerce')


In [35]:
#Visualizacion de datos
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------
# 1. Cargar dataset
# ----------------------
df = pd.read_csv("../data/accidentesLimpio.csv")

print("Dimensiones:", df.shape)
print(df.head())

# ----------------------
# 2. Crear PDF de informe
# ----------------------
with PdfPages("Informe_Accidentes.pdf") as pdf:

    # Portada
    fig, ax = plt.subplots(figsize=(9,6))
    ax.axis("off")

    ax.text(0.5, 0.7, "Informe de Accidentes Viales", fontsize=22, fontweight="bold", ha="center")
    ax.text(0.5, 0.55, "Análisis de Mortalidad y KPIs", fontsize=16, ha="center")

    ax.text(0.5, 0.35, "Autor: Daniel / Fernando / Ripariav", fontsize=12, ha="center")
    ax.text(0.5, 0.3, "Fecha: Agosto 2025", fontsize=12, ha="center")
    pdf.savefig()
    plt.close()

    # Figura 1: tendencia anual
    plt.figure(figsize=(9,6))
    df["Año"].value_counts().sort_index().plot(kind="line", marker="o")
    plt.title("Tendencia de accidentes por año")
    plt.xlabel("Año")
    plt.ylabel("Número de accidentes")
    pdf.savefig()
    plt.close()

    # Figura 2: accidentes por mes
    plt.figure(figsize=(9,6))
    df["Mes"].value_counts().sort_index().plot(kind="bar")
    plt.title("Accidentes por mes")
    plt.xlabel("Mes")
    plt.ylabel("Número de accidentes")
    pdf.savefig()
    plt.close()

    #Figura 2.1: Clase de Accidente
    plt.figure(figsize=(9,6))
    df["Clase de Accidente"].value_counts().plot(kind="pie", autopct="%1.1f%%")
    plt.title("Distribución de Clase de Accidente")
    plt.ylabel("") 
    pdf.savefig()
    plt.close()


    # Figura 3: accidentes por día
    plt.figure(figsize=(9,6))
    df["Día"].value_counts().sort_index().plot(kind="bar")
    plt.title("Accidentes por día del mes")
    plt.xlabel("Día")
    plt.ylabel("Número de accidentes")
    pdf.savefig()
    plt.close()

    # Figura 4: accidentes por barrio
    plt.figure(figsize=(9,6))
    # Filtrar los barrios que no son "NO REGISTRA"
    df_filtrado = df[df["Barrio"] != "No registra"]
    # Graficar top 15 barrios sin "NO REGISTRA"
    df_filtrado["Barrio"].value_counts().head(15).plot(kind="bar")
    plt.title("Top 15 barrios con más accidentes REGISTRADOS")
    plt.xlabel("Barrio")
    plt.ylabel("Número de accidentes")
    plt.xticks(rotation=45, ha="right", fontsize=10)
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Figura 5: pie chart muertes vs no muertes
    plt.figure(figsize=(9,6))
    df["Mortalidad_label"].value_counts().plot(
    kind="pie", autopct="%1.1f%%", startangle=90)
    plt.title("Accidentes con y sin muertes")
    plt.ylabel("") 
    pdf.savefig()
    plt.close()

    # Figura 6: KPIs
    fig, ax = plt.subplots(figsize=(9,6))
    ax.axis("off")
    # KPIs
    tasa_mortalidad = df["Muertes"].sum() / len(df)
    kpi_muertes = df["Mortalidad"].value_counts(normalize=True) * 100

    # Texto en bloques
    ax.text(0.1, 0.9, "Indicadores Clave (KPIs)", fontsize=16, fontweight="bold", va="top")

    ax.text(0.1, 0.7, f"Tasa de mortalidad por accidente:\n {tasa_mortalidad:.2%}",
            fontsize=14, bbox=dict(facecolor="#f0f0f0", boxstyle="round,pad=0.5"))

    ax.text(0.1, 0.5, f"Accidentes con muertes:\n {kpi_muertes.get(True,0):.2f}%",
            fontsize=14, bbox=dict(facecolor="#ffe0e0", boxstyle="round,pad=0.5"))

    ax.text(0.1, 0.3, f"Accidentes sin muertes:\n {kpi_muertes.get(False,0):.2f}%",
            fontsize=14, bbox=dict(facecolor="#e0ffe0", boxstyle="round,pad=0.5"))
    pdf.savefig()
    plt.close()


Dimensiones: (512, 13)
  Fecha_Ocurrencia                                          Direccion  \
0       2024-01-03                      CALLE 14A CARRERA 34   BACHUE   
1       2024-01-06                   CALLE 14 CARRERA  28   SAN JOSÃ   
2       2024-01-09                                KILOMETRO 7+800 MTS   
3       2024-01-12                    AVENIDA 23 N. 24 - 07   LA TIZA   
4       2024-01-12  KILOMETRO 3+500 MTS VIA CHICHIMENE ESTACION AC...   

        Barrio  Vehiculos Involucrados  Heridos Accidente con  Muertes  \
0       BACHUE                       2        2       HERIDOS        0   
1    SAN JOSÃ                       2        3       HERIDOS        0   
2  No registra                       1        1       HERIDOS        0   
3      LA TIZA                       2        2       HERIDOS        0   
4  No registra                       2        1       HERIDOS        0   

  Clase de Accidente  Día  Mes   Año  Mortalidad Mortalidad_label  
0             CHOQUE    3

/tmp/ipykernel_7047/1295597508.py:77: UserWarning: Glyph 137 (\x89) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_7047/1295597508.py:78: UserWarning: Glyph 137 (\x89) missing from font(s) DejaVu Sans.
  pdf.savefig()
